## Request and Inspect Weekly SPX-Options

First, load ib-ruby environment

In [1]:
require_relative '../lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

Check the Presence of predefined indices

In [2]:
Symbols::Index.all

[:a_d, :asx, :dax, :hsi, :minihsi, :spx, :stoxx, :tick, :trin, :vasx, :vdax, :vhsi, :vix, :volume, :vstoxx]

> Symbols::Index holds a selection of predefined indices

In [3]:
G.logger.level = Logger::FATAL
spx_market_price =  Symbols::Index.spx.market_price


4221.19

_Note:_ If the request is fired before the market opening, the returned market price is the closing price of the previous day.

 * Get the most recent expiry 
 * Define a range of strikes to consider as  market-price +/- 15 points.
 


In [4]:
# declare variables to be modified in the loop
spxw_options=  nil
date =  DateTime.now

loop do  
 spxw_options =  Option.new( trading_class: 'SPXW', 
                             symbol: 'SPX', 
                             currency: 'USD', 
                             expiry: date.strftime("%Y%m%d").to_i,
                             exchange: 'SMART', 
                             right: :put ).verify do |c|
      c if c.strike > spx_market_price - 15 && c.strike < spx_market_price + 15 
   end
 if spxw_options.empty?
   date = date.tomorrow
 else
   break
 end
end
  
puts spxw_options.as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ SPX    │ 467183869 │  SMART   │ 20210618 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 465903810 │  SMART   │ 20210618 │    100     │     SPXW      │  put  │ 4225.0 │   USD    │
│ Option │ SPX    │ 467183858 │  SMART   │ 20210618 │    100     │     SPXW      │  put  │ 4210.0 │   USD    │
│ Option │ SPX    │ 467183863 │  SMART   │ 20210618 │    100     │     SPXW      │  put  │ 4220.0 │   USD    │
│ Option │ SPX    │ 480705280 │  SMART   │ 20210618 │    100     │     SPXW      │  put  │ 4215.0 │   USD    │
│ Option │ SPX    │ 480705285 │  SMART   │ 20210618 │    100     │     SPXW      │  put  │ 4235.0 │   USD    │
└

Lets inspect these Options further.

The method _request_greeks_ requests Tickdata from the TWS-Server and collects the calculated greeks 


In [5]:
spxw_options.map{|c| c.request_greeks thread: true }.each &:join
puts spxw_options.greek.as_table

┌────────────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                         │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞════════════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ put  SPX/SPXW 20210618  4230.0 │   37.99 │       0.22 │     0.00 │   -0.94 │    0.01 │    0.14 │   -0.68 │
│ put  SPX/SPXW 20210618  4225.0 │   33.36 │       0.22 │     0.00 │   -0.91 │    0.01 │    0.19 │   -1.05 │
│ put  SPX/SPXW 20210618  4210.0 │   19.91 │       0.19 │     0.00 │   -0.79 │    0.01 │    0.33 │      -- │
│ put  SPX/SPXW 20210618  4220.0 │   28.86 │       0.21 │     0.00 │   -0.87 │    0.01 │    0.24 │   -1.55 │
│ put  SPX/SPXW 20210618  4215.0 │   23.99 │       0.19 │     0.00 │   -0.85 │    0.01 │    0.26 │   -1.68 │
│ put  SPX/SPXW 20210618  4235.0 │   42.50 │       0.19 │     0.00 │   -0.98 │    0.00 │    0.06 │   -0.19 │
└──────────────────

### Get ATM-Weeklies

There is a method `Contract#atm-options`. Its designed for monthly options

In [6]:
atm_options =  Symbols::Index.spx.atm_options

atm_options.keys    # Expiries of the Options

[2107, 2108, 2109, 2111, 2112, 2201, 2202, 2203, 2205, 2206, 2209, 2212, 2301, 2306]

In [7]:
puts atm_options[atm_options.keys.first].as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ SPX    │ 467200323 │  SMART   │ 20210715 │    100     │      SPX      │  put  │ 4220.0 │   USD    │
└────────┴────────┴───────────┴──────────┴──────────┴────────────┴───────────────┴───────┴────────┴──────────┘


To get all available ATM-SPXW-Options we define a sample (as above) and specify the strike

In [17]:
spxw =  Option.new trading_class: :spxw, symbol: :spx, currency: :usd, strike: 4230, 
                   exchange: :smart, right: :put

spxw_chain =  spxw.verify.sort_by &:last_trading_day
puts spxw_chain.to_human

<Option: SPX 20210618 put 4230.0 SMART USD>
<Option: SPX 20210621 put 4230.0 SMART USD>
<Option: SPX 20210623 put 4230.0 SMART USD>
<Option: SPX 20210625 put 4230.0 SMART USD>
<Option: SPX 20210628 put 4230.0 SMART USD>
<Option: SPX 20210630 put 4230.0 SMART USD>
<Option: SPX 20210702 put 4230.0 SMART USD>
<Option: SPX 20210706 put 4230.0 SMART USD>
<Option: SPX 20210707 put 4230.0 SMART USD>
<Option: SPX 20210709 put 4230.0 SMART USD>
<Option: SPX 20210712 put 4230.0 SMART USD>
<Option: SPX 20210714 put 4230.0 SMART USD>
<Option: SPX 20210716 put 4230.0 SMART USD>
<Option: SPX 20210721 put 4230.0 SMART USD>
<Option: SPX 20210723 put 4230.0 SMART USD>
<Option: SPX 20210730 put 4230.0 SMART USD>
<Option: SPX 20210806 put 4230.0 SMART USD>
<Option: SPX 20210820 put 4230.0 SMART USD>
<Option: SPX 20210831 put 4230.0 SMART USD>
<Option: SPX 20210917 put 4230.0 SMART USD>
<Option: SPX 20210930 put 4230.0 SMART USD>
<Option: SPX 20211015 put 4230.0 SMART USD>
<Option: SPX 20211029 put 4230.0

In [9]:
spxw_chain.map{|o| o.request_greeks( thread: true)}.each &:join
puts spxw_chain.greek.as_table

┌────────────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                         │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞════════════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ put  SPX/SPXW 20210618  4230.0 │   37.98 │       0.22 │     0.00 │   -0.94 │    0.01 │    0.14 │   -0.68 │
│ put  SPX/SPXW 20210621  4230.0 │   43.76 │       0.12 │     0.42 │   -0.78 │    0.01 │    1.17 │      -- │
│ put  SPX/SPXW 20210623  4230.0 │   49.16 │       0.13 │     0.48 │   -0.72 │    0.01 │    1.71 │      -- │
│ put  SPX/SPXW 20210625  4230.0 │   53.65 │       0.13 │     0.59 │   -0.68 │    0.00 │    2.11 │   -1.90 │
│ put  SPX/SPXW 20210628  4230.0 │   56.90 │       0.12 │     0.97 │   -0.67 │    0.00 │    2.56 │   -1.52 │
│ put  SPX/SPXW 20210630  4230.0 │   60.92 │       0.13 │     1.31 │   -0.65 │    0.00 │    2.85 │   -1.45 │
│ put  SPX/SPXW 202